In [1]:
import pandas as pd
import torch
from transformers import AutoModel
import json
import os
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MultiLabelBinarizer
from sentence_transformers import SentenceTransformer, InputExample, losses
from transformers import AutoTokenizer
import os
import torch

from sklearn.preprocessing import MultiLabelBinarizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the dataset class

# Load data from json file
with open('../reports/generic_epochs_200_train_size_full.json') as f:
    data = json.load(f)

dfs = []
for k, v in data.items():
    valid_metrics = v['valid']
    valid_metrics['dataset'] = 'valid'
    valid_metrics['fold'] = int(k) + 1
    dfs.append(pd.DataFrame([valid_metrics]))
    
    test_metrics = v['test']
    test_metrics['dataset'] = 'test'
    test_metrics['fold'] = int(k) + 1
    dfs.append(pd.DataFrame([test_metrics]))

# Concatenate all dataframes together
df = pd.concat(dfs, ignore_index=True)

# Rename columns
df.columns = df.columns.str.replace('eval_', '')
df = df.rename(columns={'epoch': 'num_epochs'})

# Print the final dataframe
print(df)

c:\Users\bruno\.conda\envs\my_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


   accuracy  micro_precision  micro_recall  micro_f1  macro_precision  \
0  0.965167         0.814042      0.794444  0.804124         0.791356   
1  0.962333         0.792063      0.767102  0.779383         0.740888   
2  0.961583         0.827021      0.734545  0.778045         0.844839   
3  0.964800         0.829497      0.747886  0.786580         0.787015   
4  0.965667         0.857298      0.736891  0.792548         0.831705   
5  0.964200         0.837456      0.728670  0.779285         0.790979   
6  0.964833         0.821569      0.777365  0.798856         0.784618   
7  0.964667         0.809639      0.774789  0.791830         0.742390   
8  0.964167         0.809293      0.782039  0.795433         0.726300   
9  0.963000         0.794164      0.774020  0.783963         0.722006   

   macro_recall  macro_f1  runtime  samples_per_second  steps_per_second  \
0      0.708402  0.739394    8.677              92.198            23.049   
1      0.647392  0.683971   15.968          

In [2]:
df

,accuracy,micro_precision,micro_recall,micro_f1,macro_precision,macro_recall,macro_f1,runtime,samples_per_second,steps_per_second,num_epochs,dataset,fold
0,0.965167,0.814042,0.794444,0.804124,0.791356,0.708402,0.739394,8.677,92.198,23.049,6.5,valid,1
1,0.962333,0.792063,0.767102,0.779383,0.740888,0.647392,0.683971,15.968,62.625,15.656,6.5,test,1
2,0.961583,0.827021,0.734545,0.778045,0.844839,0.615006,0.684758,9.193,87.023,21.756,5.0,valid,2
3,0.964800,0.829497,0.747886,0.786580,0.787015,0.608488,0.676405,15.502,64.508,16.127,5.0,test,2
4,0.965667,0.857298,0.736891,0.792548,0.831705,0.595904,0.678940,7.258,110.223,27.556,4.0,valid,3
5,0.964200,0.837456,0.728670,0.779285,0.790979,0.580760,0.646839,15.526,64.408,16.102,4.0,test,3
6,0.964833,0.821569,0.777365,0.798856,0.784618,0.672926,0.718014,15.658,51.092,12.773,7.0,valid,4
7,0.964667,0.809639,0.774789,0.791830,0.742390,0.667765,0.696235,21.475,46.566,11.641,7.0,test,4
8,0.964167,0.809293,0.782039,0.795433,0.726300,0.704469,0.707340,11.861,67.448,16.862,6.5,valid,5
9,0.963000,0.794164,0.774020,0.783963,0.722006,0.692502,0.696317,18.055,55.386,13.847,6.5,test,5


In [3]:
df[df.dataset == "valid"].mean()

C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\2172152441.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[df.dataset == "valid"].mean()


accuracy               0.964283
micro_precision        0.825845
micro_recall           0.765057
micro_f1               0.793801
macro_precision        0.795764
macro_recall           0.659342
macro_f1               0.705689
runtime               10.529400
samples_per_second    81.596800
steps_per_second      20.399200
num_epochs             5.800000
fold                   3.000000
dtype: float64

In [4]:
df[df.dataset == "test"].mean()

C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\952663639.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df[df.dataset == "test"].mean()


accuracy               0.963800
micro_precision        0.812564
micro_recall           0.758493
micro_f1               0.784208
macro_precision        0.756656
macro_recall           0.639382
macro_f1               0.679954
runtime               17.305200
samples_per_second    58.698600
steps_per_second      14.674600
num_epochs             5.800000
fold                   3.000000
dtype: float64

In [40]:
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModel,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    DataCollatorWithPadding,
    TrainerCallback
)
from transformers import RobertaConfig, RobertaModel
from transformers import RobertaPreTrainedModel
from typing import List, Dict

import seaborn as sns
import matplotlib.pyplot as plt

def model_summary(model):
    print("Model summary:")
    print("---------------------------")
    total_params = 0
    for name, param in model.named_parameters():
        param_count = param.numel()
        total_params += param_count
    print(f"Total parameters: {total_params}")
    
"""def print_report(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, target_names=categories)
    print(report)
    sns.heatmap(cm, annot=True, xticklabels=categories, yticklabels=categories, fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()"""

class TweetDataset(Dataset):
    def __init__(self, x, y, mlb, tokenizer):
        self.x = x
        self.y = y
        self.mlb = mlb
        self.tokenizer = tokenizer
        self.encoded_tweets = self.preprocess_text(self.x)

    
    def preprocess_text(self, text):
        return self.tokenizer(text, return_attention_mask=True, return_tensors='pt', padding=True)
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        label = self.y[idx]
        return {'input_ids': self.encoded_tweets['input_ids'][idx],
                'attention_mask': self.encoded_tweets['attention_mask'][idx],
                'label': torch.tensor(label, dtype=torch.float32)}
        
class MultiLabelDataCollator(DataCollatorWithPadding):
    def __init__(self, tokenizer):
        super().__init__(tokenizer)

    def __call__(self, features: List[Dict[str, torch.Tensor]]):
        batch = super().__call__(features)
        batch["labels"] = torch.stack([feature["label"] for feature in features])
        return batch
    
class RobertaForMultiLabelClassification(RobertaPreTrainedModel):
    def __init__(self, config, num_classes):
        super().__init__(config)
        self.roberta = RobertaModel(config)
        self.classifier = nn.Linear(config.hidden_size, num_classes)
        self.init_weights()

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        pooled_output = torch.mean(last_hidden_state, dim=1)
        logits = self.classifier(pooled_output)
        return logits
    
def calculate_average_report(reports):
    avg_report = {}
    for report in reports:
        for key, scores in report.items():
            if key not in avg_report:
                avg_report[key] = {}
                for score_key, score_value in scores.items():
                    avg_report[key][score_key] = score_value
            else:
                for score_key, score_value in scores.items():
                    avg_report[key][score_key] += score_value

    num_reports = len(reports)
    for key, scores in avg_report.items():
        for score_key in scores:
            avg_report[key][score_key] /= num_reports

    return avg_report

def calculate_metrics(task):
    k = 5
    
    classification_reports = []

    # Loop over each fold and load the corresponding model
    for fold in range(k):
        model_path = f"../models/generic_epochs_200_train_size_full_fold_{fold}"
        # find the latest checkpoint file
        checkpoint_files = [f for f in os.listdir(model_path) if f.startswith("checkpoint")]
        latest_checkpoint = os.path.join(model_path, sorted(checkpoint_files)[-1])
        
        # Load the model and tokenizer
        model = AutoModel.from_pretrained(latest_checkpoint)
        model.to(device)
        tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large")

        filename = f"../data/labeled_data/{task}_test_{fold}.json"
        with open(filename) as f:
            data = json.load(f)
        train_df = pd.DataFrame(data["train"])
        val_df = pd.DataFrame(data["valid"])
        test_df = pd.DataFrame(data["test"])
        
        train_annotations = train_df["annotations"].tolist()
        classes = set()
        for annotation in train_annotations:
            classes.update(annotation)
        classes = sorted(list(classes))
        
        model.classifier = nn.Linear(model.config.hidden_size, len(classes))
        checkpoint = torch.load(os.path.join(latest_checkpoint, "pytorch_model.bin"))
        model.load_state_dict(checkpoint)
        
        model.to(device)
        
        mlb = MultiLabelBinarizer(classes=classes)
        
        train_labels = mlb.fit_transform(train_df["annotations"])
        val_labels = mlb.transform(val_df["annotations"])
        test_labels = mlb.transform(test_df["annotations"])
        
        train_dataset = TweetDataset(train_df['text'].to_list(), torch.tensor(train_labels), mlb, tokenizer)
        val_dataset = TweetDataset(val_df['text'].to_list(), torch.tensor(val_labels), mlb, tokenizer)
        test_dataset = TweetDataset(test_df['text'].to_list(), torch.tensor(test_labels), mlb, tokenizer)
        
        test_loader = torch.utils.data.DataLoader(
            test_dataset, batch_size=4, shuffle=False, collate_fn=MultiLabelDataCollator(tokenizer)
        )
        model.eval()
        labels = []
        predictions = []
        for batch in test_loader:
            batch_inputs = {'input_ids': batch['input_ids'].to(device),
                            'attention_mask': batch['attention_mask'].to(device)}
            with torch.no_grad():
                outputs = model(**batch_inputs)
                last_hidden_state = outputs.last_hidden_state
                pooled_output = torch.mean(last_hidden_state, dim=1)
                logits = model.classifier(pooled_output)
            batch_predictions = (logits > 0.5).detach().cpu().numpy().astype(int)
            predictions.append(batch_predictions)
            labels.append(batch['labels'].detach().cpu().numpy().astype(int))

        predictions = np.concatenate(predictions, axis=0)
        labels = np.concatenate(labels, axis = 0)

        #cm = multilabel_confusion_matrix(labels, predictions)
        dict_report = classification_report(labels, predictions, target_names=classes, labels=range(len(classes)), zero_division=0, output_dict=True)
        classification_reports.append(dict_report)
        
        report = classification_report(labels, predictions, target_names=classes, labels=range(len(classes)), zero_division=0)

        print(f"Fold {fold}")
        #print(f"Micro F1 score: {micro_f1}")
        print("Classification report:")
        print(report)
        #sns.heatmap(cm, annot=True, xticklabels=classes, yticklabels=classes, fmt='g')
        #plt.xlabel('Predicted')
        #plt.ylabel('True')
        #plt.show()
    # Calculate and print the average report
    average_report = calculate_average_report(classification_reports)
    print("\nAverage classification report:")
    print(average_report)
    
calculate_metrics("generic")

Some weights of the model checkpoint at ../models/generic_epochs_200_train_size_full_fold_0\checkpoint-5200 were not used when initializing RobertaModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\2650439182.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(label, dtype=torch.float32)}
You're using a RobertaTokeniz

Fold 0
Classification report:
                                    precision    recall  f1-score   support

                 Conspiracy Theory       0.53      0.42      0.47        45
                         Education       0.56      0.69      0.62        13
                 Election Campaign       0.85      0.85      0.85        33
                       Environment       0.67      0.57      0.62        14
                 Government/Public       0.79      0.73      0.76       291
                            Health       0.73      0.65      0.69        46
           Immigration/Integration       0.81      0.69      0.75        36
                     Justice/Crime       0.83      0.85      0.84       137
                  Labor/Employment       0.56      0.64      0.60        28
Macroeconomics/Economic Regulation       0.80      0.66      0.73        62
                  Media/Journalism       0.73      0.62      0.67        48
                            Others       0.87      0.76  

Some weights of the model checkpoint at ../models/generic_epochs_200_train_size_full_fold_1\checkpoint-4000 were not used when initializing RobertaModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\2650439182.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(label, dtype=torch.float32)}
You're using a RobertaTokeniz

Fold 1
Classification report:
                                    precision    recall  f1-score   support

                 Conspiracy Theory       0.64      0.16      0.25        45
                         Education       0.50      0.69      0.58        13
                 Election Campaign       0.77      0.73      0.75        33
                       Environment       1.00      0.50      0.67        14
                 Government/Public       0.80      0.71      0.75       291
                            Health       0.81      0.63      0.71        46
           Immigration/Integration       0.95      0.56      0.70        36
                     Justice/Crime       0.83      0.77      0.80       137
                  Labor/Employment       0.74      0.50      0.60        28
Macroeconomics/Economic Regulation       0.84      0.69      0.76        62
                  Media/Journalism       0.93      0.56      0.70        48
                            Others       0.80      0.86  

Some weights of the model checkpoint at ../models/generic_epochs_200_train_size_full_fold_2\checkpoint-3200 were not used when initializing RobertaModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\2650439182.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(label, dtype=torch.float32)}
You're using a RobertaTokeniz

Fold 2
Classification report:
                                    precision    recall  f1-score   support

                 Conspiracy Theory       0.56      0.49      0.52        45
                         Education       0.38      0.38      0.38        13
                 Election Campaign       0.88      0.64      0.74        33
                       Environment       0.50      0.64      0.56        14
                 Government/Public       0.66      0.89      0.76       291
                            Health       0.74      0.61      0.67        46
           Immigration/Integration       0.76      0.86      0.81        36
                     Justice/Crime       0.73      0.87      0.80       137
                  Labor/Employment       0.72      0.46      0.57        28
Macroeconomics/Economic Regulation       0.85      0.53      0.65        62
                  Media/Journalism       0.87      0.56      0.68        48
                            Others       0.94      0.71  

Some weights of the model checkpoint at ../models/generic_epochs_200_train_size_full_fold_3\checkpoint-5600 were not used when initializing RobertaModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\2650439182.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(label, dtype=torch.float32)}
You're using a RobertaTokeniz

Fold 3
Classification report:
                                    precision    recall  f1-score   support

                 Conspiracy Theory       0.63      0.42      0.51        45
                         Education       0.42      0.62      0.50        13
                 Election Campaign       0.81      0.88      0.84        33
                       Environment       0.89      0.57      0.70        14
                 Government/Public       0.73      0.77      0.75       291
                            Health       0.76      0.54      0.63        46
           Immigration/Integration       0.74      0.64      0.69        36
                     Justice/Crime       0.67      0.88      0.76       137
                  Labor/Employment       0.60      0.64      0.62        28
Macroeconomics/Economic Regulation       0.77      0.60      0.67        62
                  Media/Journalism       0.81      0.62      0.71        48
                            Others       0.86      0.77  

Some weights of the model checkpoint at ../models/generic_epochs_200_train_size_full_fold_4\checkpoint-5200 were not used when initializing RobertaModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\bruno\AppData\Local\Temp\ipykernel_17768\2650439182.py:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'label': torch.tensor(label, dtype=torch.float32)}
You're using a RobertaTokeniz

Fold 4
Classification report:
                                    precision    recall  f1-score   support

                 Conspiracy Theory       0.54      0.42      0.48        45
                         Education       0.56      0.69      0.62        13
                 Election Campaign       0.88      0.85      0.86        33
                       Environment       0.62      0.57      0.59        14
                 Government/Public       0.78      0.78      0.78       291
                            Health       0.73      0.65      0.69        46
           Immigration/Integration       0.80      0.67      0.73        36
                     Justice/Crime       0.89      0.74      0.81       137
                  Labor/Employment       0.68      0.46      0.55        28
Macroeconomics/Economic Regulation       0.87      0.65      0.74        62
                  Media/Journalism       0.79      0.71      0.75        48
                            Others       0.83      0.83  